In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import quad
from ipywidgets import interact, FloatSlider


## Physics Behind the Work Computation

This simulation models the work done on a golf ball flying through the air under two forces:

- **Gravitational Force**:  
  $$ \vec{F}_g = -mg \hat{z} $$

- **Air Resistance** (linear in velocity):  
  $$ \vec{F}_d = -\gamma \vec{v} $$

### Work Done

- By gravity:  
  $$ W_g = \int -mg \frac{dz}{dt} \, dt $$

- By air resistance (velocity-dependent):  
  $$ W_d = \int \vec{F}_d \cdot \vec{v} \, dt = -\gamma \int |\vec{v}(t)|^2 \, dt $$

### Parametric Paths

1. **Linear**:  
   $$ \vec{r}(t) = (10t, 20t, 10t), \quad t \in [0, 1] $$

2. **Polynomial**:  
   $$ \vec{r}(t) = (10t, 20t, -20t^2 + 30t), \quad t \in [0, 1] $$

3. **Circular (x–z arc)**:  
   $$ \vec{r}(t) = \left(10 \sin\left( \frac{\pi t}{2} \right),\ 20t,\ 10\left(1 - \cos\left( \frac{\pi t}{2} \right) \right)\right), \quad t \in [0, 1] $$


In [7]:

# Constants
m = 0.045  # mass of a golf ball (kg)
g = 9.81   # gravity (m/s^2)

# Start and end points
A = np.array([0, 0, 0])
B = np.array([10, 20, 10])

# Parametric paths
def linear_path(t):
    return np.array([10 * t, 20 * t, 10 * t])

def polynomial_path(t):
    return np.array([10 * t, 20 * t, -20 * t**2 + 30 * t])

def circular_path(t):
    R = 10
    theta = (np.pi / 2) * t
    x = R * np.sin(theta)
    y = 20 * t
    z = R * (1 - np.cos(theta))
    return np.array([x, y, z])
def compute_work(path_type, gamma):
    if path_type == 'linear':
        dx_dt = lambda t: 10
        dy_dt = lambda t: 20
        dz_dt = lambda t: 10

    elif path_type == 'polynomial':
        dx_dt = lambda t: 10
        dy_dt = lambda t: 20
        dz_dt = lambda t: -40 * t + 30

    elif path_type == 'circular':
        R = 10
        dtheta_dt = np.pi / 2
        dx_dt = lambda t: R * np.cos(dtheta_dt * t) * dtheta_dt
        dy_dt = lambda t: 20
        dz_dt = lambda t: R * np.sin(dtheta_dt * t) * dtheta_dt

    else:
        raise ValueError("Invalid path type")

    v_squared = lambda t: dx_dt(t)**2 + dy_dt(t)**2 + dz_dt(t)**2
    gravitational = lambda t: -m * g * dz_dt(t)
    drag = lambda t: -gamma * v_squared(t)

    if path_type == 'circular':
        return quad(lambda t: gravitational(t) + drag(t), 0, 1)[0]
    else:
        return quad(lambda t: gravitational(t) + drag(t), 0, 1)[0]

# Visualization function
def plot_paths_and_work(gamma):
    t_vals_lin = np.linspace(0, 1, 100)
    t_vals_poly = np.linspace(0, 1, 100)
    t_vals_circ = np.linspace(0, 1, 100)

    path_lin = np.array([linear_path(t) for t in t_vals_lin])
    path_poly = np.array([polynomial_path(t) for t in t_vals_poly])
    path_circ = np.array([circular_path(t) for t in t_vals_circ])

    fig = plt.figure(figsize=(10, 6))
    ax = fig.add_subplot(111, projection='3d')

    ax.plot(path_lin[:, 0], path_lin[:, 1], path_lin[:, 2], label='Linear Path')
    ax.plot(path_poly[:, 0], path_poly[:, 1], path_poly[:, 2], label='Polynomial Path')
    ax.plot(path_circ[:, 0], path_circ[:, 1], path_circ[:, 2], label='Circular Path (x-z arc)')

    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z (Height)')
    ax.set_title(f'Paths under Gravity and Air Resistance (gamma = {gamma:.2f})')
    ax.legend()
    plt.show()

    work_lin = compute_work('linear', gamma)
    work_poly = compute_work('polynomial', gamma)
    work_circ = compute_work('circular', gamma)

    print(f"Work Done (Gravity + Air Resistance) for gamma = {gamma:.2f}:")
    print(f"  Linear Path:     {work_lin:.4f} J")
    print(f"  Polynomial Path: {work_poly:.4f} J")
    print(f"  Circular Path:   {work_circ:.4f} J")

# Interactive widget to explore different gamma values
interact(plot_paths_and_work, gamma=FloatSlider(value=0.0, min=0.0, max=0.5, step=0.01, description='gamma'));

   

interactive(children=(FloatSlider(value=0.0, description='gamma', max=0.5, step=0.01), Output()), _dom_classes…